# AutoEncoders

## Downloading the dataset

### ML-100K

In [ ]:
# !wget "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
# !unzip ml-100k.zip
# !ls

### ML-1M

In [ ]:
# !wget "http://files.grouplens.org/datasets/movielens/ml-1m.zip"
# !unzip ml-1m.zip
# !ls

## Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

## Importing the dataset


In [ ]:
# We won't be using this dataset.
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

## Preparing the training set and the test set


In [2]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

## Getting the number of users and movies


In [3]:
nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))

## Converting the data into an array with users in lines and movies in columns


In [4]:
def convert(data):
  new_data = []
  for id_users in range(1, nb_users + 1):
    id_movies = data[:, 1] [data[:, 0] == id_users]
    id_ratings = data[:, 2] [data[:, 0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)

## Converting the data into Torch tensors


In [5]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

## Creating the architecture of the Neural Network


In [6]:
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
    def __call__(self, x):
        return self.forward(x)


In [7]:
sae = SAE()
criterion = nn.MSELoss()
a = 0.001
optimizer = optim.RMSprop(sae.parameters(), lr = a, weight_decay = 0.5)

    

## Training the SAE


In [10]:
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data*mean_corrector)
            optimizer.step()
            s = s + 1
    print('epoch: ' + str(epoch) + ' loss: ' + str(train_loss/s))


epoch: 1 loss: tensor(1.8563)
epoch: 2 loss: tensor(1.0921)
epoch: 3 loss: tensor(1.0485)
epoch: 4 loss: tensor(1.0334)
epoch: 5 loss: tensor(1.0259)
epoch: 6 loss: tensor(1.0215)
epoch: 7 loss: tensor(1.0186)
epoch: 8 loss: tensor(1.0167)
epoch: 9 loss: tensor(1.0153)
epoch: 10 loss: tensor(1.0142)
epoch: 11 loss: tensor(1.0134)
epoch: 12 loss: tensor(1.0128)
epoch: 13 loss: tensor(1.0123)
epoch: 14 loss: tensor(1.0119)
epoch: 15 loss: tensor(1.0116)
epoch: 16 loss: tensor(1.0114)
epoch: 17 loss: tensor(1.0112)
epoch: 18 loss: tensor(1.0110)
epoch: 19 loss: tensor(1.0108)
epoch: 20 loss: tensor(1.0107)
epoch: 21 loss: tensor(1.0106)
epoch: 22 loss: tensor(1.0105)
epoch: 23 loss: tensor(1.0104)
epoch: 24 loss: tensor(1.0103)
epoch: 25 loss: tensor(1.0103)
epoch: 26 loss: tensor(1.0102)
epoch: 27 loss: tensor(1.0101)
epoch: 28 loss: tensor(1.0101)
epoch: 29 loss: tensor(1.0101)
epoch: 30 loss: tensor(1.0100)
epoch: 31 loss: tensor(1.0100)
epoch: 32 loss: tensor(1.0100)
epoch: 33 loss: t

## Testing the SAE


In [11]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies / \
            float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data*mean_corrector)
        s = s + 1
print('test loss: ' + str(test_loss/s))


test loss: tensor(0.9299)
